In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import visdom

In [ ]:
torch.manual_seed(1)
device='cuda'if torch.cuda.is_available()else'cpu'
if device=='cuda':
    torch.cuda.manual_seed_all(1)

In [ ]:
vis=visdom.Visdom()

In [ ]:
def loss_track(loss_plot,loss_value,num):
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append')

In [ ]:
mnist_train=dsets.MNIST(root='./mnist_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.MNIST(root='./mnist_data',
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                        shuffle=True,
                                        batch_size=100,
                                        drop_last=True,
                                        num_workers=2)

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #(100,1,28,28)
        self.layer1=torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2))
        #(100,32,14,14)
        self.layer2=torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2))
        #(100,64,7,7)
        self.layer3=torch.nn.Sequential(
        torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=1))
        #(100,128,4,4)
        self.fc1=torch.nn.Linear(128*4*4,625)
        self.relu=torch.nn.ReLU()
        self.fc2=torch.nn.Linear(625,10)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.fc1(out.view(-1,128*4*4))
        out=self.relu(out)
        out=self.fc2(out)
        return out

In [ ]:
model=CNN().to(device)

value=(torch.Tensor(1,1,28,28)).to(device)
print(model(value).shape)

In [ ]:
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=0.003)

In [ ]:
loss_plt=vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker',legend=['loss'],showlegend=True))

In [ ]:
#training
total_batch = len(data_loader)

for epoch in range(10):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_track(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())